In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [26]:
don=pd.read_csv("../1_data/private/VitalantData.csv") #with hgb values

In [7]:
pd.set_option('display.max_columns', 100)
don.head()

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,PHLEB_STOP_TIME_KEY,PHLEBOTOMY_CODE,PHLEBOTOMY_DESC,PHLEBOTOMY_STATUS,DONATION_PROCEDURE_TYPE,PHLEBOTOMY_GROUP,DONATION_TYPE,DONATION_BAG_DESCRIPTION,DONOR_GENDER,DONOR_ABORH,DONOR_ETHNICITY_DESC,RACE_ETHNICITY,RACE_DESCRIPTION,DONOR_COUNTRY_OF_BIRTH,DONOR_EDUCATION_DESC,DONOR_WEIGHT,DONOR_HEIGHT,DONOR_ZIPCODE,DONOR_AGE_AT_DONATION,COLLECTION_FIXED_LOCATION,COLLECTION_SITE_VALID_FLAG,BECS_ID,DRIVE_SETUP_TYPE,DRIVE_TYPE_FIXED_MOBILE,DEFERRAL_EVENT,DEFERRAL_INFORMATION,DEFERRAL_CREATE_DATE,DEFERRAL_START_DATE,DEFERRAL_STOP_DATE,DEF_INVALDATION_DATE,DEFERRAL,DEFERRAL_DESCRIPTION,DEFERRAL_VALID_CODE,DEFERRAL_DURATION,DURATION_MODIF_TYPE,ALLOW_DISTRIBUTION,DEFERRAL_GROUP,MQ_ANSWER_x,MQ_ANSWER_y,MQ_ANSWER_x.1,MQ_ANSWER_y.1,SITE_STATE,ACCOUNT_TYPE_NAME,SITE_ZIPCode,Visit_Date,DonorID,cum_lifetime_donations,unit_rbc_loss,time_to_fu,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss
0,20130625.0,15057895.0,82X912T,W28861300166600F,Active,3003.0,62130.0,NaN,NaN,18:58:01,19:10:01,W,WHOLE BLOOD,Successful Phlebotomy,Other,Whole Blood,Allogeneic,Historical Triple Bag,M,O +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),BACHELOR'S DEGREE,278.0,74.0,606,56.0,Yes,Y,VTL,Fixed,Fixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-06-25T00:00:00Z,15057895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20140525.0,840652.0,5006735,W04221402205100P,First Time,51.0,383.0,500.0,612.0,17:04:01,17:14:01,W,WHOLE BLOOD,Successful Phlebotomy,WB,Whole Blood,Allogeneic,CP2D: Triple RBC Filter,M,O +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),SOME COLLEGE OR TECHNICAL SCHOOL,175.0,69.0,890,63.0,No,Y,VTL,Mobile Setup,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.8,NaN,15.8,NaN,NaN,NaN,NaN,2014-05-25T00:00:00Z,840652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20141018.0,792640.0,2109508,W04221404264200Q,Active,51.0,376.0,452.0,600.0,10:24:01,12:14:01,A,APHERESIS (PLT/GRAN),Successful Phlebotomy,Automated,Platelet,Allogeneic,TRIMA APHERESIS PLATELETS,M,AB+,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),SOME COLLEGE OR TECHNICAL SCHOOL,205.0,68.0,890,36.0,Yes,Y,VTL,Fixed,Fixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6,NaN,16.6,NaN,PA,Healthcare,152.0,2014-10-18T00:00:00Z,792640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20140726.0,7079135.0,5024441,W04221402021200I,First Time,51.0,380.0,500.0,602.0,12:28:01,12:49:01,U,UNSUCCESSFUL DRAW,Unsuccessful Phlebotomy,WB,Whole Blood,Allogeneic,CP2D: Triple RBC Filter,M,O +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),HIGH SCHOOL GRAD-DIPLOMA OR EQUIV,190.0,70.0,891,25.0,Yes,Y,VTL,Fixed,Fixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.3,NaN,16.3,NaN,NaN,NaN,NaN,2014-07-26T00:00:00Z,7079135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20140519.0,6126680.0,4186033,W042214021267009,Active,51.0,383.0,440.0,612.0,11:31:01,12:05:01,R,2RBC - ALLOGENEIC,Successful Phlebotomy,Automated,2RBC,Allogeneic,HAEMONETICS APHERESIS 2RBC FILTERED,M,O +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),"9th,10th,11th,12th GRADE-NO DIPLOMA",190.0,69.0,864,18.0,No,Y,VTL,Mobile Setup,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.1,NaN,16.1,NaN,NaN,NaN,NaN,2014-05-19T00:00:00Z,6126680.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# a function to compute deferral rates and make a 2X2 table

def make_table(df):
    first=df.query("first_time == 1")
    repeat=df.query("first_time == 0")
    
    #create a dict
    table={}
    table['-']=['Female', 'Male', 'All']

        #first time
    female=(len(first.loc[(first.DONOR_GENDER == 'F') & (first.OUTCOME_TYPE=='low hgb')])/len(first.loc[(first.DONOR_GENDER == 'F')]))
    male=(len(first.loc[(first.DONOR_GENDER == 'M') & (first.OUTCOME_TYPE=='low hgb')])/len(first.loc[(first.DONOR_GENDER == 'M')]))
    alls=(len(first.loc[(first.OUTCOME_TYPE=='low hgb')])/len(first))
    table['First Time']=[female, male, alls]

        #repeat
    female=(len(repeat.loc[(repeat.DONOR_GENDER == 'F') & (repeat.OUTCOME_TYPE=='low hgb')])/len(repeat.loc[(repeat.DONOR_GENDER == 'F')]))
    male=(len(repeat.loc[(repeat.DONOR_GENDER == 'M') & (repeat.OUTCOME_TYPE=='low hgb')])/len(repeat.loc[(repeat.DONOR_GENDER == 'M')]))
    alls=(len(repeat.loc[(repeat.OUTCOME_TYPE=='low hgb')])/len(repeat))
    table['Repeat']=[female, male, alls]

        #all 
    female=(len(df.loc[(df.DONOR_GENDER == 'F') & (df.OUTCOME_TYPE=='low hgb')])/len(df.loc[(df.DONOR_GENDER == 'F')]))
    male=(len(df.loc[(df.DONOR_GENDER == 'M') & (df.OUTCOME_TYPE=='low hgb')])/len(df.loc[(df.DONOR_GENDER == 'M')]))
    alls=(len(df.loc[(df.OUTCOME_TYPE=='low hgb')])/len(df))
    table['All']=[female, male, alls]
    table=pd.DataFrame(table)
    
    return table
    


In [28]:
don['HGB_value']=np.where((don.MQ_ANSWER_y.notna()),don.MQ_ANSWER_y, don.MQ_ANSWER_x)# if hgb recheck avaible use that as hgb val else use hgb val

don['HGB_value']=pd.to_numeric(don['HGB_value'], errors='coerce')

#convert MQ_answer to float
don['HGB_value']=pd.to_numeric(don['HGB_value'], errors='coerce')
#replace all nan hgb values with 999
don['HGB_value'].replace(np.nan, 999, regex=True, inplace=True)
don.loc[(don['HGB_value']<13.0) & (don['DONOR_GENDER'] == "M"), 'OUTCOME_TYPE'] = 'low hgb'
don.loc[(don['HGB_value']<12.5) & (don['DONOR_GENDER'] == "F"), 'OUTCOME_TYPE'] = 'low hgb'
don['first_time']=np.where(don.ACTIVE_DONOR_TYPE=='First Time', 1,0)
table=make_table(don)
print(table)
don['OUTCOME_TYPE'].value_counts()

        -  First Time    Repeat       All
0  Female    0.006113  0.006356  0.006315
1    Male    0.002583  0.004755  0.004448
2     All    0.004399  0.005502  0.005333


low hgb    84785
Name: OUTCOME_TYPE, dtype: int64

In [32]:
df=don.sample(100000)

In [33]:
df[['DONOR_GENDER', 'DONATION_VOLUME_DRAWN', 'HGB_value', 'OUTCOME_TYPE', 'MQ_ANSWER_x','MQ_ANSWER_y']].loc[
    (df['OUTCOME_TYPE']=='low hgb') & (df['DONOR_GENDER']== 'M')].head()

,DONOR_GENDER,DONATION_VOLUME_DRAWN,HGB_value,OUTCOME_TYPE,MQ_ANSWER_x,MQ_ANSWER_y
14388212,M,-1.0,12.7,low hgb,12.7,NaN
11591935,M,500.0,12.8,low hgb,12.8,NaN
8270280,M,-1.0,11.9,low hgb,12.3,11.9
2484220,M,920.0,12.6,low hgb,12.6,NaN
9506139,M,940.0,12.8,low hgb,12.8,NaN


In [36]:
df.loc[(df['DONATION_VOLUME_DRAWN']==-1.0)& (df['DONOR_GENDER']== 'M')].head(50)

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,PHLEB_STOP_TIME_KEY,PHLEBOTOMY_CODE,PHLEBOTOMY_DESC,PHLEBOTOMY_STATUS,DONATION_PROCEDURE_TYPE,PHLEBOTOMY_GROUP,DONATION_TYPE,DONATION_BAG_DESCRIPTION,DONOR_GENDER,DONOR_ABORH,DONOR_ETHNICITY_DESC,RACE_ETHNICITY,RACE_DESCRIPTION,DONOR_COUNTRY_OF_BIRTH,DONOR_EDUCATION_DESC,DONOR_WEIGHT,DONOR_HEIGHT,DONOR_ZIPCODE,DONOR_AGE_AT_DONATION,COLLECTION_FIXED_LOCATION,COLLECTION_SITE_VALID_FLAG,BECS_ID,DRIVE_SETUP_TYPE,DRIVE_TYPE_FIXED_MOBILE,DEFERRAL_EVENT,DEFERRAL_INFORMATION,DEFERRAL_CREATE_DATE,DEFERRAL_START_DATE,DEFERRAL_STOP_DATE,DEF_INVALDATION_DATE,DEFERRAL,DEFERRAL_DESCRIPTION,DEFERRAL_VALID_CODE,DEFERRAL_DURATION,DURATION_MODIF_TYPE,ALLOW_DISTRIBUTION,DEFERRAL_GROUP,MQ_ANSWER_x,MQ_ANSWER_y,MQ_ANSWER_x.1,MQ_ANSWER_y.1,SITE_STATE,ACCOUNT_TYPE_NAME,SITE_ZIPCode,Visit_Date,DonorID,cum_lifetime_donations,unit_rbc_loss,time_to_fu,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss,HGB_value,OUTCOME_TYPE,first_time
15889561,20121120.0,3379269.0,3671793,W041012195024001,Active,26.0,26.0,-1.0,NaN,17:08:01,17:08:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,B +,"NOT HISPANIC - BLACK, AFRICAN AMERICAN","Black, NH",NaN,UNITED STATES (INCLUDING HAWAII),BACHELOR'S DEGREE,270.0,71.0,851,45.0,Yes,Y,VTL,Fixed,Fixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-20T00:00:00Z,3379269.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,NaN,0
14388212,20140221.0,944614.0,4345898,W04161400179000A,First Time,39.0,199.0,-1.0,NaN,15:24:01,15:24:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,UNKNOWN,"NOT HISPANIC - BLACK, AFRICAN AMERICAN","Black, NH",NaN,UNITED STATES (INCLUDING HAWAII),"9th,10th,11th,12th GRADE-NO DIPLOMA",NaN,NaN,393,16.0,No,Y,VTL,Mobile,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7,NaN,12.7,NaN,NaN,NaN,NaN,2014-02-21T00:00:00Z,944614.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7,low hgb,1
8626684,20120808.0,2916240.0,4117390,W04101210949500F,First Time,26.0,15.0,-1.0,NaN,18:24:01,18:24:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,UNKNOWN,NOT HISPANIC - WHITE,"White, NH",NaN,CANADA,SOME COLLEGE OR TECHNICAL SCHOOL,NaN,NaN,850,28.0,No,Y,VTL,Mobile,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-08-08T00:00:00Z,2916240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,NaN,1
8270280,20130327.0,3875970.0,3067851,W042413011424002,Active,55.0,444.0,-1.0,NaN,14:41:01,14:41:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,O +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),SOME COLLEGE OR TECHNICAL SCHOOL,175.0,69.0,577,62.0,Yes,Y,VTL,Fixed,Fixed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.3,11.9,12.3,11.9,NaN,NaN,NaN,2013-03-27T00:00:00Z,3875970.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.9,low hgb,0
2837745,20131227.0,4117203.0,3999479,W04251308589200Y,Active,108.0,532.0,-1.0,NaN,15:49:01,15:49:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,A +,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),SOME COLLEGE OR TECHNICAL SCHOOL,NaN,NaN,580,18.0,No,Y,VTL,Mobile Setup,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.4,NaN,17.4,NaN,NaN,NaN,NaN,2013-12-27T00:00:00Z,4117203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.4,NaN,0
13432452,20121025.0,3674624.0,0636603,W04161204941800D,Active,39.0,228.0,-1.0,NaN,14:00:01,14:00:01,*,NOT DRAWN,No Phlebotomy,ND,Not Drawn,Allogeneic,NO BAG USED,M,O -,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),BACHELOR'S DEGREE,264.0,67.0,397,37.0,No,N,VTL,Mobile Setup,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-10-25T00:00:00Z,3674624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,NaN,0
6077906,20130605.0,252386.0,4016401,W04101309185500Y,Active,26.0,32.0,-1.0,N

In [37]:
def_columns=['DEFERRAL_EVENT', 'DEFERRAL_INFORMATION',
       'DEFERRAL_CREATE_DATE', 'DEFERRAL_START_DATE', 'DEFERRAL_STOP_DATE',
       'DEF_INVALDATION_DATE', 'DEFERRAL', 'DEFERRAL_DESCRIPTION',
       'DEFERRAL_VALID_CODE', 'DEFERRAL_DURATION', 'DURATION_MODIF_TYPE',
       'ALLOW_DISTRIBUTION', 'DEFERRAL_GROUP']

df['def_isna']=df[def_columns].isnull().apply(lambda x: all(x), axis=1) 


In [39]:
df.loc[(df['def_isna']==False)& (df['OUTCOME_TYPE']== 'low hgb')].head(50)

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,PHLEB_STOP_TIME_KEY,PHLEBOTOMY_CODE,PHLEBOTOMY_DESC,PHLEBOTOMY_STATUS,DONATION_PROCEDURE_TYPE,PHLEBOTOMY_GROUP,DONATION_TYPE,DONATION_BAG_DESCRIPTION,DONOR_GENDER,DONOR_ABORH,DONOR_ETHNICITY_DESC,RACE_ETHNICITY,RACE_DESCRIPTION,DONOR_COUNTRY_OF_BIRTH,DONOR_EDUCATION_DESC,DONOR_WEIGHT,DONOR_HEIGHT,DONOR_ZIPCODE,DONOR_AGE_AT_DONATION,COLLECTION_FIXED_LOCATION,COLLECTION_SITE_VALID_FLAG,BECS_ID,DRIVE_SETUP_TYPE,DRIVE_TYPE_FIXED_MOBILE,DEFERRAL_EVENT,DEFERRAL_INFORMATION,DEFERRAL_CREATE_DATE,DEFERRAL_START_DATE,DEFERRAL_STOP_DATE,DEF_INVALDATION_DATE,DEFERRAL,DEFERRAL_DESCRIPTION,DEFERRAL_VALID_CODE,DEFERRAL_DURATION,DURATION_MODIF_TYPE,ALLOW_DISTRIBUTION,DEFERRAL_GROUP,MQ_ANSWER_x,MQ_ANSWER_y,MQ_ANSWER_x.1,MQ_ANSWER_y.1,SITE_STATE,ACCOUNT_TYPE_NAME,SITE_ZIPCode,Visit_Date,DonorID,cum_lifetime_donations,unit_rbc_loss,time_to_fu,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss,HGB_value,OUTCOME_TYPE,first_time,def_isna
11838015,20210513.0,1824920.0,0284821,W04102106500000C,Non allogeneic donor,26.0,26.0,500.0,152.0,12:14:01,12:20:01,T,THERAPEUTIC PHLEBOTOMY,Successful Phlebotomy,WB,Whole Blood,Therapeutic,Therapeutic - NO ADDITIVE,M,A -,Other,"Other/Mixed, NH",NaN,UNITED STATES (INCLUDING HAWAII),NaN,185.0,69.0,852,68.0,Yes,Y,VTL,Fixed,Fixed,Medical Quest. Consequence,Donation W04102106500000C Question FW Answer P...,2021-05-13,2021-05-13 00:00:00,2295-02-25 00:00:00,9998-12-30 00:00:00,3019.0,"BLOOD DISEASE/BLEED DISORDER, PERMANENT",Y,99999.0,End Date Can be Lengthened or Shortened (user ...,Don't allow,Medical Condition,12.0,NaN,12.0,NaN,NaN,NaN,NaN,2021-05-13T00:00:00Z,1824920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,low hgb,0,False
3912347,20190814.0,3681172.0,3957929,W04191902449700Q,Non allogeneic donor,47.0,298.0,525.0,10.0,08:57:01,09:04:01,T,THERAPEUTIC PHLEBOTOMY,Successful Phlebotomy,WB,Whole Blood,Therapeutic,CP2D: Triple RBC Filter,M,UNKNOWN,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),BACHELOR'S DEGREE,245.0,75.0,894,67.0,Yes,Y,VTL,Fixed,Fixed,Medical Quest. Consequence,Donation W04191902449700Q Question FW Answer P...,2019-08-14,2019-08-14 00:00:00,2293-05-28 00:00:00,9998-12-30 00:00:00,3019.0,"BLOOD DISEASE/BLEED DISORDER, PERMANENT",Y,99999.0,End Date Can be Lengthened or Shortened (user ...,Don't allow,Medical Condition,12.2,NaN,12.2,NaN,NaN,NaN,NaN,2019-08-14T00:00:00Z,3681172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.2,low hgb,0,False
435506,20180724.0,938964.0,82UB92R,W04111804968400U,Non allogeneic donor,30.0,93.0,500.0,302.0,12:49:01,12:56:01,T,THERAPEUTIC PHLEBOTOMY,Successful Phlebotomy,WB,Whole Blood,Therapeutic,CP2D: Double WB Filter Set,M,UNKNOWN,NOT HISPANIC - WHITE,"White, NH",NaN,UNITED STATES (INCLUDING HAWAII),MASTER'S OR PROFESSIONAL DEGREE,174.0,67.0,880,65.0,Yes,Y,VTL,Fixed,Fixed,Medical Quest. Consequence,Donation W04111804968400U Question FW Answer P...,2018-07-24,2018-07-24 00:00:00,2292-05-07 00:00:00,9998-12-30 00:00:00,3019.0,"BLOOD DISEASE/BLEED DISORDER, PERMANENT",Y,99999.0,End Date Can be Lengthened or Shortened (user ...,Don't allow,Medical Condition,12.5,NaN,12.5,NaN,NaN,NaN,NaN,2018-07-24T00:00:00Z,938964.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5,low hgb,0,False
